In [1]:
import feature_extractor
import flow_estimation
import torch
from functools import partial
import torch.nn as nn

In [2]:
def init_model_config(F=32, W=7, depth=[2, 2, 2, 4, 4]):
    '''This function should not be modified'''
    return { 
        'embed_dims':[F, 2*F, 4*F, 8*F, 16*F],
        'motion_dims':[0, 0, 0, 8*F//depth[-2], 16*F//depth[-1]],
        'num_heads':[8*F//32, 16*F//32],
        'mlp_ratios':[4, 4],
        'qkv_bias':True,
        'norm_layer':partial(nn.LayerNorm, eps=1e-6), 
        'depths':depth,
        'window_sizes':[W, W]
    }, {
        'embed_dims':[F, 2*F, 4*F, 8*F, 16*F],
        'motion_dims':[0, 0, 0, 8*F//depth[-2], 16*F//depth[-1]],
        'depths':depth,
        'num_heads':[8*F//32, 16*F//32],
        'window_sizes':[W, W],
        'scales':[4, 8, 16],
        'hidden_dims':[4*F, 4*F],
        'c':F
    }

MODEL_CONFIG = {
    'LOGNAME': 'ours',
    'MODEL_TYPE': (feature_extractor, flow_estimation),
    'MODEL_ARCH': init_model_config(
        F = 32,
        W = 7,
        depth = [2, 2, 2, 4, 4]
    )
}

In [3]:
imgs = torch.randn(8,3,224,224).cuda()
backbonecfg, multiscalecfg = init_model_config(F=32, W=7, depth=[2, 2, 2, 4, 4])
net = flow_estimation.MultiScaleFlow(feature_extractor.MotionFormer(**backbonecfg), **multiscalecfg).cuda()
flow, pred = net(imgs)
total = sum([param.nelement() for param in net.parameters()])
print("Number of parameters: %.2fM" % (total/1e6))

torch.Size([4, 512, 14, 14]) torch.Size([4, 512, 14, 14])
1 torch.Size([4, 2048, 14, 14]) torch.Size([4, 6, 224, 224])
1_ torch.Size([4, 128, 56, 56]) torch.Size([4, 6, 224, 224])
torch.Size([4, 6, 224, 224])
2 16 torch.Size([4, 6, 56, 56])
3 torch.Size([4, 128, 56, 56]) torch.Size([4, 6, 56, 56])
torch.Size([4, 256, 28, 28]) torch.Size([4, 256, 28, 28])
1 torch.Size([4, 1024, 28, 28]) torch.Size([4, 13, 224, 224])
1_ torch.Size([4, 64, 112, 112]) torch.Size([4, 13, 224, 224])
torch.Size([4, 13, 224, 224])
2 8 torch.Size([4, 13, 112, 112])
3 torch.Size([4, 64, 112, 112]) torch.Size([4, 17, 112, 112])
torch.Size([4, 3, 224, 224]) torch.Size([4, 3, 224, 224]) torch.Size([4, 3, 224, 224])


ValueError: too many values to unpack (expected 2)